In [4]:
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import regularizers
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, Callback

%matplotlib inline

In [3]:
## define the network structure and train the network

model = Sequential()

# Normalize
model.add(Lambda(lambda x: x/127.5 - 1.0,input_shape=(66,200,3)))

# Add three 5x5 convolution layers (output depth 24, 36, and 48), each with 2x2 stride
model.add(Conv2D(24, (5, 5), strides=(2, 2), padding='valid', kernel_regularizer=regularizers.l2(0.001)))
model.add(ELU())
model.add(Conv2D(36, (5, 5), strides=(2, 2), padding='valid', kernel_regularizer=regularizers.l2(0.001)))
model.add(ELU())
model.add(Conv2D(48, (5, 5), strides=(2, 2), padding='valid', kernel_regularizer=regularizers.l2(0.001)))
model.add(ELU())

#model.add(Dropout(0.50))

# Add two 3x3 convolution layers (output depth 64, and 64)
model.add(Conv2D(64, (3, 3), padding='valid', kernel_regularizer=regularizers.l2(0.001)))
model.add(ELU())
model.add(Conv2D(64, (3, 3), padding='valid', kernel_regularizer=regularizers.l2(0.001)))
model.add(ELU())

# Add a flatten layer
model.add(Flatten())

# Add three fully connected layers (depth 100, 50, 10), tanh activation (and dropouts)
model.add(Dense(100, kernel_regularizer=regularizers.l2(0.001)))
model.add(ELU())
#model.add(Dropout(0.50))
model.add(Dense(50, kernel_regularizer=regularizers.l2(0.001)))
model.add(ELU())
#model.add(Dropout(0.50))
model.add(Dense(10, kernel_regularizer=regularizers.l2(0.001)))
model.add(ELU())
#model.add(Dropout(0.50))

# Add a fully connected output layer
model.add(Dense(1))

# Compile and train the model, 
#model.compile('adam', 'mean_squared_error')
model.compile(optimizer=Adam(lr=1e-4), loss='mse')


In [4]:
def crop_roi(image):
    img = image[55:121, 50:250, :]
    return img

def preprocess_image(image):
    # crop the ROI
    img = crop_roi(image)
    
    # Apply colorspace conversion from BGR to YUV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
       
    return img

def read_data(csv_path):
    train_df = pd.read_csv(csv_path, names=['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed'])
    train_df['steering_left'] = train_df['steering'] + 0.25
    train_df['steering_right'] = train_df['steering'] - 0.25
    
    img_data = np.array(train_df['center'])
    img_data = np.append(img_data, train_df['left'])
    img_data = np.append(img_data, train_df['right'])
    
    angle_labels = np.array(train_df['steering'])
    angle_labels = np.append(angle_labels, train_df['steering_left'])
    angle_labels = np.append(angle_labels, train_df['steering_right'])
    
    return img_data, angle_labels, train_df
#     for index, row in train_df.iterrows():
#         img_left.append(preprocess_image(cv2.imread(row['left'])))
#         img_center.append(preprocess_image(cv2.imread(row['center'])))
#         img_right.append(preprocess_image(cv2.imread(row['right'])))
    
#     return train_df

def generate_training_data(batch_size, data, labels):
    num_samples = data.size
    
    while True:
        indxs = np.random.choice(num_samples, batch_size, replace=True)

        batch_data, batch_labels = data[indxs], labels[indxs]
        batch_imgs = []

        for img_path in batch_data:
            batch_imgs.append(preprocess_image(cv2.imread(img_path)))
        
        yield (np.array(batch_imgs), batch_labels)
    
    
def get_image_data(data):
    img_data = []
    
    for img_path in data:
        img_data.append(preprocess_image(cv2.imread(img_path)))
    
    return np.array(img_data)
    

data, labels, train_df = read_data('/Users/dhruvsharma/Desktop/4B/driving_log.csv')
data, labels = shuffle(data, labels)
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = 0.2, random_state=42)

train_gen = generate_training_data(64, train_data, train_labels)
test_img_data = get_image_data(test_data)


In [6]:
checkpoint = ModelCheckpoint('model_{epoch:02d}.h5', period=5)

In [ ]:
model.fit_generator(train_gen, epochs=25, validation_data=(test_img_data,test_labels), verbose=1, steps_per_epoch=340, callbacks=[checkpoint])